In [1]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import numpy as np

from utils.plots import visualize_vehicle_trajectories, plot_interactive_matrix
from utils.loader import load_data_from_database
from utils.transformer import categorize_ids

In [2]:
print("Do you want to load data from the database? This may take a while.")
if input("Type 'y' to proceed: ").lower() == 'y':
    df = load_data_from_database()
    df = df.sort_values(['date_time', 'frame_id'])
else:
    df = pd.read_csv("raw_traffic_data.csv", parse_dates=['date_time'])
    df = df.sort_values(['date_time', 'frame_id'])

Do you want to load data from the database? This may take a while.


In [3]:
df['session_id'] = (
    (df['frame_id'].diff() < 0)
).cumsum()

In [4]:
df.groupby('session_id').agg('size')

session_id
0       28090
1       15936
2     8238814
3       33537
4        7082
5       10582
6        8504
7       17186
8       30834
9        5361
10      26281
11      17976
12      12914
13      26303
14     489001
15      17536
dtype: int64

In [35]:
# 1. Select one stable session and sort
session_id_to_analyze = 2
session_df = df[df['session_id'] == session_id_to_analyze].sort_values(['vehicle_id', 'date_time'])

# Filter vehicle_id with less than 20 records
# vehicle_counts = session_df['vehicle_id'].value_counts()
# valid_vehicles = vehicle_counts[vehicle_counts >= 20].index
# session_df = session_df[session_df['vehicle_id'].isin(valid_vehicles)]


In [36]:
session_df

,id,vehicle_id,area,x,y,width,heigth,date_time,frame_id,session_id
8941910,44027,0,2307.5,22,79,36.0,104.0,2025-11-29 18:02:55.716052,2,2
8941909,44028,1,1490.5,21,139,26.0,76.0,2025-11-29 18:02:56.049113,6,2
8941908,44029,2,1835.0,15,156,29.0,85.0,2025-11-29 18:02:56.348841,11,2
8941907,44030,3,1073.5,126,266,52.0,24.0,2025-11-29 18:04:13.894244,1280,2
8941906,44031,3,1306.0,131,255,46.0,35.0,2025-11-29 18:04:13.939439,1281,2
...,...,...,...,...,...,...,...,...,...,...
703101,8282836,404155,1218.0,138,0,43.0,30.0,2025-12-18 11:02:47.723669,42975019,2
703100,8282837,404155,1040.0,138,0,41.0,27.0,2025-12-18 11:02:47.779042,42975020,2
703099,8282838,404155,858.0,138,0,40.0,23.0,2025-12-18 11:02:47.828348,42975021,2
703098,8282839,404155,819.0,139,0,40.0,22.0,2025-12-18 11:02:47.868994,42975022,2


In [37]:
import pandas as pd
import numpy as np

def classify_tracks(metrics):
    """
    Classifies tracks based on geometric and temporal metrics.
    """
    # Calculate movement efficiency (path per frame)
    # This helps identify objects that were stationary (Static) at any point in the ROI
    metrics['movement_efficiency'] = metrics['path_completeness'] / metrics['frames_count']

    # --- Classification conditions ---

    # 1. GHOST: Technical noise (very short tracks)
    is_ghost = (metrics['frames_count'] < 10)

    # 2. STATIC: Stationary object (at start, end, or in traffic jam)
    # If there's too little movement per frame
    is_static = (metrics['movement_efficiency'] < 0.0015) | \
                ((metrics['frames_count'] > 200) & (metrics['path_completeness'] < 0.3))

    # 3. PERFECT: Ideal passage (stable width, full path, normal speed)
    is_perfect = (
        (metrics['path_completeness'] > 0.85) & 
        (metrics['w_cv'] < 0.30) & 
        (metrics['movement_efficiency'] >= 0.0015)
    )

    # 4. ENTRY/EXIT: Full passages where height changed regularly (ID 153, 238, etc.)
    is_entry_exit = (
        (metrics['path_completeness'] > 0.85) & 
        (metrics['w_cv'] < 0.30) & 
        (metrics['h_cv'] > 0.35)
    )

    # 5. FLICKERING: Unstable object (strong width jumps)
    is_flickering = (metrics['w_cv'] > 0.45)

    # 6. PARTIAL: Stable fragments (vehicles that appeared/disappeared mid-frame)
    is_partial = (
        (metrics['path_completeness'].between(0.3, 0.85)) & 
        (metrics['w_cv'] < 0.30)
    )

    # Priority order (from most important/simplest to general)
    conditions = [
        is_ghost,
        is_static,
        is_perfect,
        is_entry_exit,
        is_flickering,
        is_partial
    ]

    choices = [
        'Ghost', 
        'Static', 
        'Perfect', 
        'EntryExit', 
        'Flickering', 
        'Partial'
    ]

    # All others become candidates for merging (RelayCandidate)
    metrics['category'] = np.select(conditions, choices, default='RelayCandidate')
    
    return metrics

def categorize_ids(df):
    """
    Aggregates raw data into metrics for each vehicle and classifies them.
    """
    # Convert time and sort for calculation stability
    df['date_time'] = pd.to_datetime(df['date_time'])
    df = df.sort_values(['session_id', 'date_time', 'frame_id'])

    # Group by session and ID
    grouped = df.groupby(['session_id', 'vehicle_id'])
    
    # ROI height (according to your settings 460 - 170)
    ROI_H = 290 
    
    # Data aggregation
    metrics = grouped.agg(
        y_start=('y', 'first'),
        y_end=('y', 'last'),
        w_mean=('width', 'mean'),
        w_std=('width', 'std'),
        h_mean=('heigth', 'mean'),
        h_std=('heigth', 'std'),
        frames_count=('frame_id', 'count'),
        t_start=('date_time', 'min'),
        t_end=('date_time', 'max'),
        x_mean=('x', 'mean'),
        x_std=('x', 'std')
    ).reset_index()

    # Calculate path completeness (0.0 - 1.0)
    metrics['path_completeness'] = (metrics['y_end'] - metrics['y_start']).abs() / ROI_H

    # Calculate size stability (Coefficient of Variation)
    # Use fillna(0) for single-frame objects
    metrics['w_cv'] = (metrics['w_std'] / metrics['w_mean']).fillna(0)
    metrics['h_cv'] = (metrics['h_std'] / metrics['h_mean']).fillna(0)

    # Run classification
    final_summary = classify_tracks(metrics)
    
    return final_summary

final_summary = categorize_ids(session_df)

In [38]:
final_summary.groupby('category').agg('size')

category
Flickering           579
Ghost             267187
Partial            23217
Perfect            28458
RelayCandidate     41487
Static             43229
dtype: int64

In [47]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.stats import gaussian_kde

def plot_interactive_matrix(final_summary, dims=None, width=1200, height=1100, 
                           category_filter=None, max_points_per_category=None):
    """
    Creates an interactive matrix plot with KDE curves on diagonal and scatter plots on off-diagonal.
    
    Parameters:
    -----------
    final_summary : DataFrame
        Summary DataFrame with vehicle metrics and categories
    dims : list
        List of dimension names to plot (default: ['path_completeness', 'frames_count', 'movement_efficiency', 'w_cv', 'h_cv'])
    width : int
        Figure width in pixels (default: 1200)
    height : int
        Figure height in pixels (default: 1100)
    category_filter : list or None
        List of category names to display (default: None - all categories)
    max_points_per_category : int or None
        Maximum number of points to display per category (default: None - all points)
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        The generated figure
    """
    
    # Default dimensions
    if dims is None:
        dims = ['path_completeness', 'frames_count', 'movement_efficiency', 'w_cv', 'h_cv']
    
    # Filter by categories if specified
    if category_filter is not None:
        final_summary = final_summary[final_summary['category'].isin(category_filter)]
    
    # Sample data if max_points_per_category is specified
    if max_points_per_category is not None:
        sampled_dfs = []
        for cat, group in final_summary.groupby('category'):
            sampled = group.sample(min(len(group), max_points_per_category), random_state=42)
            sampled_dfs.append(sampled)
        final_summary = pd.concat(sampled_dfs, ignore_index=True)
    
    categories = final_summary['category'].unique()
    n = len(dims)

    # Create subplot grid
    fig = make_subplots(
        rows=n, cols=n, 
        shared_xaxes=False, shared_yaxes=False,
        horizontal_spacing=0.03, vertical_spacing=0.03,
        column_titles=dims, row_titles=dims
    )

    # Color palette
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

    # Fill the matrix
    for i, y_col in enumerate(dims):
        for j, x_col in enumerate(dims):
            for k, cat in enumerate(categories):
                df_sub = final_summary[final_summary['category'] == cat]
                
                # If not enough data for KDE (less than 2 points), skip the curve
                if len(df_sub) < 2:
                    continue

                if i == j:  # DIAGONAL: Smooth KDE curves
                    # Calculate KDE
                    x_range = np.linspace(final_summary[x_col].min(), final_summary[x_col].max(), 100)
                    try:
                        kde = gaussian_kde(df_sub[x_col])
                        y_kde = kde(x_range)
                        
                        fig.add_trace(
                            go.Scatter(
                                x=x_range, y=y_kde, 
                                name=cat, line=dict(color=colors[k % len(colors)], width=2),
                                fill='tozeroy', opacity=0.3,  # Fill under the curve
                                showlegend=(i == 0 and j == 0),
                                legendgroup=cat
                            ),
                            row=i+1, col=j+1
                        )
                    except:
                        pass  # In case of zero variance

                else:  # OFF-DIAGONAL: Scatter plots
                    fig.add_trace(
                        go.Scatter(
                            x=df_sub[x_col], y=df_sub[y_col],
                            mode='markers', name=cat, marker_color=colors[k % len(colors)],
                            opacity=0.5, marker_size=4,
                            showlegend=False, legendgroup=cat,
                            hovertext=df_sub['vehicle_id'].apply(lambda x: f"ID: {x}")
                        ),
                        row=i+1, col=j+1
                    )

    # Configure fixed axes
    for i, col in enumerate(dims):
        margin = (final_summary[col].max() - final_summary[col].min()) * 0.05
        r = [final_summary[col].min() - margin, final_summary[col].max() + margin]
        
        for k in range(1, n + 1):
            fig.update_xaxes(range=r, row=k, col=i+1)
            if i != k-1:  # Don't touch Y axis for diagonal, as it has density scale
                fig.update_yaxes(range=r, row=i+1, col=k)

    fig.update_layout(
        title_text="Interactive matrix with distribution curves (KDE) on diagonal",
        width=width, height=height,
        template="plotly_white"
    )

    return fig

fig = plot_interactive_matrix(final_summary, max_points_per_category=500)
fig.show()


In [ ]:
# filtered_final_summary = final_summary[(final_summary['category'] == 'RelayCandidate')]
# filtered_final_summary

In [ ]:
# ids_to_plot = filtered_final_summary['vehicle_id']
# plot_data = session_df[session_df['vehicle_id'].isin(ids_to_plot)]
# visualize_vehicle_trajectories(plot_data, session_id=session_id_to_analyze, max_vehicles=25, min_records=20, category='RelayCandidate')

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

def find_merging_pairs(summary_df, time_gap_limit=1.5, space_gap_limit=40, size_sim_limit=0.2):
    """
    summary_df: result of metrics calculation (final_summary)
    time_gap_limit: max time gap (seconds)
    space_gap_limit: max distance between points (pixels)
    size_sim_limit: max width difference (relative, 0.2 = 20%)
    """
    
    # Work only with Relay candidates
    candidates = summary_df[(summary_df['category'] == 'RelayCandidate') | (summary_df['category'] == 'Static') | (summary_df['category'] == 'Ghost')].copy()
    
    # Sort by appearance time
    candidates = candidates.sort_values('t_start')
    
    merges = []
    used_as_new = set()  # Track IDs that are already used as new_id

    # Convert to list of dictionaries for fast iteration
    records = candidates.to_dict('records')

    for i in tqdm(range(len(records)), desc="Search for pairs for merging", unit="ID"):
        id_a = records[i]
        # Allow id_a even if it was previously a new_id (enables chains)

        for j in range(i + 1, len(records)):
            id_b = records[j]
            # Prevent id_b from being merged multiple times as new_id
            if id_b['vehicle_id'] in used_as_new: continue
            
            # 1. Session check (must be in the same session)
            if id_a['session_id'] != id_b['session_id']: continue

            # 2. Time gap (A ended, B started soon after)
            gap_time = (id_b['t_start'] - id_a['t_end']).total_seconds()

            # OPTIMISATION: stop the internal cycle if the time limit has been exceeded
            if gap_time > (time_gap_limit + 2.0):
                break
            
            # We're looking for B that comes AFTER A, but not later than the limit
            if -1.0 <= gap_time <= time_gap_limit:
                
                # 3. Spatial proximity (end of A to start of B)
                # Use Y as it's the main axis of movement
                dist_y = abs(id_b['y_start'] - id_a['y_end'])
                dist_x = abs(id_b['x_mean'] - id_a['x_mean'])
                
                # 4. Size similarity (width shouldn't jump)
                size_diff = abs(id_a['w_mean'] - id_b['w_mean']) / id_a['w_mean']

                if dist_y < space_gap_limit and dist_x < 20 and size_diff < size_sim_limit:
                    merges.append({
                        'old_id': int(id_a['vehicle_id']),
                        'new_id': int(id_b['vehicle_id']),
                        'gap_sec': round(gap_time, 2),
                        'y_dist': round(dist_y, 1),
                        'size_diff_pct': round(size_diff * 100, 1)
                    })
                    # Mark id_b as used, but id_b can still be old_id for next merge
                    # This enables chains: 1->2, 2->3, 3->4
                    used_as_new.add(id_b['vehicle_id'])
                    break 

    return pd.DataFrame(merges)

# Function call
merge_results = find_merging_pairs(final_summary)
print(f"Found pairs for merging: {len(merge_results)}")
merge_results.head()


Search for pairs for merging: 100%|██████████| 351903/351903 [00:01<00:00, 289281.88ID/s]


Found pairs for merging: 161880
   old_id  new_id  gap_sec  y_dist  size_diff_pct
0       1       2     0.30      17           11.5
1       5       6     0.19       0            5.1
2       6       7     0.12       0            3.6
3       7       8     0.13       0            0.4
4       8       9     0.18       0            5.3


In [ ]:
# df['year'] = df['date_time'].dt.year
# df['month'] = df['date_time'].dt.month
# df['day'] = df['date_time'].dt.day
# df['hour'] = df['date_time'].dt.hour
# df['unique_vehicle_id'] = round(df['vehicle_id'] + df['year']/10000 + df['month']/1000000 + df['day']/100000000 + df['hour']/10000000000, 10)
# df['unique_frame_id'] = round(df['vehicle_id'] + df['year']/10000 + df['month']/1000000 + df['day']/100000000 + df['hour']/10000000000, 10)
# df.head()

In [ ]:
# df['size'] = df['width'] * df['heigth']
# df.head()

In [ ]:
# min_max_df = df.groupby('unique_vehicle_id')[['size', 'y']].agg(['max', 'min']).sort_index()

In [ ]:
# min_max_df['way_size'] = min_max_df['y']['max'] - min_max_df['y']['min']
# min_max_df.columns = ['size_max', 'size_min', 'y_max', 'y_min', 'way_size']
# min_max_df.head()

In [ ]:
# min_max_df['full_way'] = min_max_df['way_size'] > 240

In [ ]:
# min_max_df['day'] = min_max_df.index.map(lambda x: True if 6 <= int((x* 10000000000)%100) < 18 else False)

In [ ]:
# min_max_df